# Chapter 2 Derivatives and Gradients

## Using SymEngine to calculate symbolic derivative.
* SymEngine
* @vars x
* f = x^2 + 2x
* diff(f,x)

It support multi-variable cases.

In [1]:
using SymEngine

In [2]:
@vars x
@vars y
f = x^2 + x/2 - sin(x)/x + y^2 + 1/y

(1/2)*x - sin(x)/x + x^2 + y^(-1) + y^2

In [3]:
println(diff(f, y))
println(diff(f, x))

2*y - y^(-2)
1/2 + 2*x + sin(x)/x^2 - cos(x)/x


## Automatic Differentiation
Automatic Differentiation is based on the application of **Chain Rule**. Complex algorithms can be decomposited to **Primitive Operations**, and these primitive operations form a **Computational Graph**. Automatic differentiation first constructs computational graph, then calculates derivates based on it. It is important for **OR** and **ML**.

This exercise tries to implement simple Forward/Reverse accumulation.
* Forward Accumulation needs one pass to calculate $\frac{\partial f}{\partial x}$, and need $n$ time pass to calculate $\nabla f$
* Reverse Accumulation is preffered when calculating gradient. It need one *forward pass* to get intermediate values, and one *backward pass* to get the gradient. Since intermediate values are saved, memory should be optimized if computational graph is large.

Automatic Differentiation Packages:
* Forward: `ForwardDiff.jl`
* Reverse: `Zygote.jl`

### Forward Accumulation

> **Keyword: Dual Number**

* Need Dual Number:
    * save variable value
    * save its derivate
* Implement Basic Operation Methods for Dual Number $\partial$

Notes:
* need to implement every possible operations with every data types
* numerical method, only calculate when values for variable and partial variable are given

In [4]:
# Dual Number
struct Dual
    v
    ∂
end

# Basic Operations: +-*/^
Base.:+(a::Dual, b::Dual) = Dual(a.v + b.v, a.∂ + b.∂)
Base.:-(a::Dual, b::Dual) = Dual(a.v - b.v, a.∂ - b.∂)
Base.:*(a::Dual, b::Dual) = Dual(a.v*b.v, a.∂*b.v + a.v*b.∂)
Base.:*(a::Int64, b::Dual) = Dual(a*b.v, a*b.∂)
Base.:/(a::Dual, b::Dual) = Dual(a.v/b.v, a.∂/b.v - a.v*b.∂/b.v^2)
Base.:^(a::Dual, b::Int64) = Dual(a.v^b, b*a.v^(b-1)*a.∂)

# define a function as expression, and eva() it
f = :(3x^2 + 2x + x*y)

x = Dual(2, 1) # ∂f/∂x
y = Dual(3, 0)

∂f∂x = eval(f).∂

x = Dual(2, 0)
y = Dual(3, 1) # ∂f/∂y

∂f∂y = eval(f).∂

fv = eval(f).v

println("For f(x, y) = 3x^2 + 2x +x*y when x = 2 and y = 3:")
println("f = $fv")
println("∂f/∂x = $∂f∂x")
println("∂f/∂y = $∂f∂y")

For f(x, y) = 3x^2 + 2x +x*y when x = 2 and y = 3:
f = 22
∂f/∂x = 17
∂f/∂y = 2


### Reverse Accumulation

> **Keyword: Recursion**

Reverse Accumulation is based on the observation, that $∂f/∂x = ∂f/∂p \cdot ∂p/∂x$. This means, at the bottom level of the computational graph, when we need to get the partial derivative, it is only related to $x$'s parent $p$. The middle part, $∂f/∂p$, can be shared with other variables.

And we can do this **recursively**, $∂f/∂p = ∂f/∂p^* \cdot ∂p^*/∂p$ **{recursion rule}**, to get all intermediate partial derivatives ($p^*$ is the parent of $p$). At the top of the graph, we have $∂f/∂f = 1$ **{base case}**.

> This is the reverse of common partial derivative calculation process, we can consider it as first expand all partial derivates along the graph, then collapse between $f$ and $p$.

The Reverse Accumulation has two phases. We can still use **Dual Number** to calculate, but the meaning is different.

In **Forward Pass** we calculate from variables to function return value, get the `Dual.v` for each intermediate variable. Besides, we can get local derivate information `Dual.∂` for each variable. `Dual.∂` is only related to variable's parent and cussion. For example $w = log(z), z = x \cdot y$:
```
x
  \
   * -- z -- log(z) -- w
  /
y

```
$x_{dual} = (x, ∂z/∂x) = (x, y)$  
$y_{dual} = (y, ∂z/∂y) = (y, x)$  
$z_{dual} = (z, ∂w/∂z) = (z, \frac{1}{z}) = (x \cdot y, \frac{1}{x \cdot y})$  
$w_{dual} = (w, ∂w/∂w) = (w, 1) = (log(x \cdot y), 1)$

#### Comparison with Forward Accumulatin:

Let $x$ be the variable, $w$ be the current intermediate partial derivate (can also equals to $x$), $p$ be the parent of $w$.

Forward: $w_{dual} = (w.value, ∂w/∂x)$  
Reverse: $w_{dual} = (w.value, ∂p/∂w)$

Forward dual value is used to calculate parent's partial derivate/dual value. Reverse dual value is used for back propagation to get the chained product.

In forward accumulation, every calculation is related to the variable $x$ you have choosen in advance. In reverse accumulation, the variable $x$ is only involved until the last level on the graph in backward pass.

The implementation of Reverse Accumulatin is more complex than Forward Accumulation. Here is an amazing blog by [Roger Luo](http://blog.rogerluo.me/).